# 도서 베스트셀러 예측을 통한 판매량 증진 전략 

## 세팅 

In [11]:
import time
import openai
from tqdm import tqdm
import json
import pandas as pd
import os 
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [12]:
def get_response_gpt(system_message, user_message, model_name, temperature, max_tokens, top_p, use_json_format=False, retry_attempts=3, delay=5):
    if use_json_format:
        json_format = {"type": "json_object"}

    messages = [
        {
            "role": "system",
            "content": system_message,
        },
        {
            "role": "user",
            "content": user_message,
        },
    ]

    attempts = 0
    while attempts < retry_attempts:
        try:
            if use_json_format:
                response = openai.ChatCompletion.create(
                    model=model_name, 
                    messages=messages,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=top_p,
                    response_format=json_format,
                )
            else:
                response = openai.ChatCompletion.create(
                    model=model_name, 
                    messages=messages,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=top_p,
                )

            return response['choices'][0]['message']['content']
        except openai.error.InvalidRequestError as e:
            error_message = str(e)
            print(f"Invalid request error: {error_message}")
            return None
        except openai.error.APIError as e:
            print(f"API error on attempt {attempts+1}: {e}")
            attempts += 1
            time.sleep(delay)

    print("Max retries reached, skipping this data.")
    return None

# 데이터

In [13]:
df = pd.read_csv('book_info.csv')
df

,번호,제목,이전 베스트셀러 작품 여부,출간일,초기 판매량(부),미디어 노출,마케팅 지원,사전 주문량(부),최근 한줄평
0,22,희망의 밥상,있음,2024-06-12,7000,있음,있음,5000,"[""너무 감동적이에요."", ""삶의 희망을 얻었어요."", ""강력 추천합니다."", ""또..."
1,23,직장인의 삶,있음,2024-06-15,8000,있음,있음,6000,"[""내용이 지루해요."", ""추천하지 않아요."", ""실망스러워요."", ""다시 안 볼 ..."
2,24,시간 여행자,있음,2024-06-16,6500,있음,있음,5500,"[""삶의 질이 높아졌어요."", ""정말 유익한 책이에요."", ""다시 읽고 싶은 책입니..."
3,25,그런 기록,없음,2024-06-18,200,없음,없음,300,"[""별로에요."", ""다시 안 볼 것 같아요."", ""좋지 않아요."", ""실망스러워요...."
4,26,빛의 자리,없음,2024-06-20,350,없음,있음,1200,"[""감동적입니다."", ""추천합니다."", ""너무 좋아요."", ""다시 읽고 싶어요."",..."
5,27,마케팅 기법,없음,2024-06-21,400,없음,있음,800,"[""내용이 지루해요."", ""추천하지 않아요."", ""실망스러워요."", ""다시 안 볼 ..."
6,28,심장,있음,2024-06-22,7200,있음,있음,5300,"[""너무 감동적이에요."", ""삶의 희망을 얻었어요."", ""강력 추천합니다."", ""또..."


# CoT 

In [43]:
system_message = """[베스트셀러 점수 계산 및 분석 방법]
다음은 도서 베스트셀러 예측 및 분석 방법입니다.
아래 항목 각각에 대해 해당하는지 확인하고, 해당하는 경우 베스트셀러 가능성 점수에 1점을 더합니다.
(1) 유명 작가(이전 작품이 베스트셀러인 경우)
(2) 출간 후 1주일 이내에 5000부 이상 판매된 경우
(3) 긍정적인 리뷰(최근 한줄평 10개 중 긍정 비율이 50% 이상인 경우)
(4) 미디어 노출(언론, TV, 라디오 등에서 다룬 경우)
(5) 출판사의 마케팅 지원(광고, 프로모션 등)
(6) 높은 사전 주문량(출간 전에 3000부 이상 주문된 경우)

아래 항목에 대해 해당하면 베스트셀러 가능성 점수에서 1점을 뺍니다.
(1) 부정적인 리뷰(최근 한줄평 10개 중 긍정 비율이 50% 미만인 경우)
(2) 낮은 초기 판매량(출간 후 1주일 이내에 500부 이하 판매된 경우)

[예시1]
아래 내용은 도서 정보입니다. 이 정보를 바탕으로 베스트셀러 점수를 산정하여 베스트셀러 가능성을 예측해주세요. 그리고 베스트셀러 가능성이 낮은 경우(4점 이하) 판매량 증진 전략을 설명해주세요.
{
    '번호': 8,
    '제목': '행복한 삶의 비결',
    '이전 베스트셀러 작품 여부': '있음',
    '출간일': '2024-06-01',
    '초기 판매량(부)': 6000,
    '미디어 노출': '있음',
    '마케팅 지원': '있음',
    '사전 주문량(부)': 4000,
    '최근 한줄평': '["정말 유익한 책이에요.", "삶의 질이 높아졌어요.", "다시 읽고 싶은 책입니다.", "친구에게 선물했어요.", "내용이 알차요.", "추천합니다.", "삶의 방향을 잡아줍니다.", "감동적입니다.", "또 읽고 싶어요.", "강추합니다."]'
}

점수 계산 및 분석: 점수 계산 및 분석: 이전 베스트셀러 작품이 있으므로 베스트셀러 가능성 점수 1점, 출간 후 1주일 이내에 6000부가 판매되어 베스트셀러 가능성 점수 1점,
최근 한줄평 10개 중 긍정 비율이 높아 베스트셀러 가능성 점수 1점, 미디어 노출이 있어 베스트셀러 가능성 점수 1점,
마케팅 지원이 있어 베스트셀러 가능성 점수 1점, 사전 주문량이 4000부로 베스트셀러 가능성 점수 1점으로 총 6점 베스트셀러 가능성 점수가 있습니다.
이 책은 베스트셀러가 될 가능성이 매우 높습니다.


[예시2]
아래 내용은 도서 정보입니다. 이 정보를 바탕으로 베스트셀러 점수를 산정하여 베스트셀러 가능성을 예측해주세요. 그리고 베스트셀러 가능성이 낮은 경우(4점 이하) 판매량 증진 전략을 설명해주세요.
{
    '번호': 23,
    '제목': '어둠의 속삭임',
    '이전 베스트셀러 작품 여부': '없음',
    '출간일': '2024-06-01',
    '초기 판매량(부)': 400,
    '미디어 노출': '없음',
    '마케팅 지원': '없음',
    '사전 주문량(부)': 200,
    '최근 한줄평': '["내용이 지루해요.", "추천하지 않아요.", "전 재밌게 봤어요.", "다시 안 볼 것 같아요.", "별로에요.", "좋지 않아요.", "생각보다 별로였어요.", "기대 이하에요.", "재미없어요.", "괜찮아요."]'
}

점수 계산 및 분석:이전 베스트셀러 작품이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다. 출간 후 1주일 이내에 400부가 판매되어 베스트셀러 가능성 점수에서 1점 감소,
최근 한줄평 10개 중 부정 비율이 높아 베스트셀러 가능성 점수에서 1점 감소,
미디어 노출이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다. 마케팅 지원이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다.
사전 주문량이 200부로 베스트셀러 가능성 점수는 증가하지 않습니다. 총 베스트셀러 가능성 점수는 -2점입니다.
이 책은 베스트셀러가 될 가능성이 낮습니다. 따라서, 판매량 증진을 위한 마케팅 전략이 필요합니다.

판매량 증진 전략은 다음과 같습니다.
(1) 작가와의 만남 및 사인회 개최
(2) 소셜 미디어를 통한 홍보 캠페인 진행
(3) 독서 모임 및 서평 이벤트 개최
(4) 할인 행사 및 프로모션 진행
(5) 언론 및 미디어와의 협력을 통한 노출 확대
(6) 긍정적인 리뷰 유도 및 리뷰어와의 협력 강화
"""

In [45]:
user_message = """아래 내용은 도서 정보입니다. 이 정보를 바탕으로 베스트셀러 점수를 산정하여 베스트셀러 가능성을 예측해주세요. 그리고 베스트셀러 가능성이 낮은 경우(4점 이하) 판매량 증진 전략을 설명해주세요.
{'번호': 26,
 '제목': '빛의 자리',
 '이전 베스트셀러 작품 여부': '없음',
 '출간일': '2024-06-20',
 '초기 판매량(부)': 350,
 '미디어 노출': '없음',
 '마케팅 지원': '있음',
 '사전 주문량(부)': 1200,
 '최근 한줄평': '["감동적입니다.", "추천합니다.", "너무 좋아요.", "다시 읽고 싶어요.", "친구들에게도 추천했어요.", "재밌어요.", "유익해요.", "최고입니다.", "삶의 질이 높아졌어요.", "강추합니다."]'}

 점수 계산 및 분석: """

In [47]:
temperature = 0.5
max_tokens = 4096
top_p = 0.5

result = get_response_gpt(system_message, user_message, model_name='gpt-4o', temperature=temperature, max_tokens=max_tokens, top_p=top_p, use_json_format=False, retry_attempts=3, delay=5)
print(result)

점수 계산 및 분석:
1. 유명 작가(이전 작품이 베스트셀러인 경우): 없음 (0점)
2. 출간 후 1주일 이내에 5000부 이상 판매된 경우: 350부 판매로 해당 없음 (0점)
3. 긍정적인 리뷰(최근 한줄평 10개 중 긍정 비율이 50% 이상인 경우): 긍정적인 리뷰가 10개 중 10개로 100% 긍정 비율 (1점)
4. 미디어 노출(언론, TV, 라디오 등에서 다룬 경우): 없음 (0점)
5. 출판사의 마케팅 지원(광고, 프로모션 등): 있음 (1점)
6. 높은 사전 주문량(출간 전에 3000부 이상 주문된 경우): 1200부로 해당 없음 (0점)

감점 항목:
1. 부정적인 리뷰(최근 한줄평 10개 중 긍정 비율이 50% 미만인 경우): 해당 없음 (0점)
2. 낮은 초기 판매량(출간 후 1주일 이내에 500부 이하 판매된 경우): 350부 판매로 해당 (1점 감점)

총 점수: 1점 (긍정적인 리뷰) + 1점 (마케팅 지원) - 1점 (낮은 초기 판매량) = 1점

이 책은 베스트셀러가 될 가능성이 낮습니다. 따라서, 판매량 증진을 위한 마케팅 전략이 필요합니다.

판매량 증진 전략:
1. **작가와의 만남 및 사인회 개최**: 독자들과의 직접적인 소통을 통해 책에 대한 관심을 높입니다.
2. **소셜 미디어를 통한 홍보 캠페인 진행**: 인스타그램, 페이스북, 트위터 등 다양한 소셜 미디어 플랫폼을 활용하여 책을 홍보합니다.
3. **독서 모임 및 서평 이벤트 개최**: 독서 모임을 통해 책에 대한 입소문을 유도하고, 서평 이벤트를 통해 긍정적인 리뷰를 확보합니다.
4. **할인 행사 및 프로모션 진행**: 한정 기간 동안 할인 행사를 진행하여 초기 구매를 유도합니다.
5. **언론 및 미디어와의 협력을 통한 노출 확대**: 언론 인터뷰, TV 프로그램 출연 등을 통해 책의 인지도를 높입니다.
6. **긍정적인 리뷰 유도 및 리뷰어와의 협력 강화**: 리뷰어들에게 책을 제공하고, 긍정적인 리뷰를 유도합니다.


# Prompt Chaining이 필요한 이유
1. 명확한 작업 흐름 : 단계별로 명확하게 정의된 작업 흐름을 통해 각 단계의 역할을 명확히 하고, 전체 프로세스를 체계적으로 관리
    - 판단해야 할 정보가 추가되는 경우: 서브 태스크로 분할 가능 
    - 유연한 대응: 상황에 따라 업무 순서가 달라질 수 있는 경우나 태스크를 더 세부적으로 쪼갤 때 등
2. 문제 해결의 정확성: 중간 결과를 확인하고, 필요에 따라 수정 및 보완하여 최종 결과의 정확성을 높일 수 있음. 
3. 자동화 및 확장 : 지금은 모든 프롬프트 내용을 수동으로 입력했지만 현업에서는 프롬프트 내에서 변수화하는 부분이 존재할 수 있음. 또한 DB에서 정보를 불러오는 부분이 필요할 수도 있음.

# CoT + Prompt Chaining 

## 질의 의도 분석

In [20]:
system_message = "사용자 쿼리에 책의 베스트셀러 점수 계산 및 분석에 대한 내용이 존재할 경우 부차적인 말 없이 도서명을 반환하세요."
user_message = f"[사용자 쿼리]\n빛의 자리라는 책의 베스트셀러 점수를 산정하여 베스트셀러 가능성을 예측해주세요. 그리고 베스트셀러 가능성이 낮은 경우(4점 이하) 판매량 증진 전략을 설명해주세요."

In [21]:
temperature = 0.5
max_tokens = 4096
top_p = 0.5

book_title = get_response_gpt(system_message, user_message, model_name='gpt-4o', temperature=temperature, max_tokens=max_tokens, top_p=top_p, use_json_format=False, retry_attempts=3, delay=5)
print(book_title)

빛의 자리


## DB에서 책 정보 가져오기
(여기에서는 간단히 예시를 보여주기 위해 DB라고 가정하고 pandas 데이터셋에서 정보를 가져옴.)

In [22]:
def get_book_info(book_title):
    df = pd.read_csv('book_info.csv')
    book_info = df[df["제목"] == book_title].iloc[0].to_dict()
    return book_info

book_info = get_book_info(book_title) 
book_info

{'번호': 26,
 '제목': '빛의 자리',
 '이전 베스트셀러 작품 여부': '없음',
 '출간일': '2024-06-20',
 '초기 판매량(부)': 350,
 '미디어 노출': '없음',
 '마케팅 지원': '있음',
 '사전 주문량(부)': 1200,
 '최근 한줄평': '["감동적입니다.", "추천합니다.", "너무 좋아요.", "다시 읽고 싶어요.", "친구들에게도 추천했어요.", "재밌어요.", "유익해요.", "최고입니다.", "삶의 질이 높아졌어요.", "강추합니다."]'}

## 베스트셀러 점수 예측 및 데이터 분석 (CoT)

In [30]:
system_message = """[베스트셀러 점수 계산 및 분석 방법]
다음은 도서 베스트셀러 예측 및 분석 방법입니다.
아래 항목 각각에 대해 해당하는지 확인하고, 해당하는 경우 베스트셀러 가능성 점수에 1점을 더합니다.
(1) 유명 작가(이전 작품이 베스트셀러인 경우)
(2) 출간 후 1주일 이내에 5000부 이상 판매된 경우
(3) 긍정적인 리뷰(최근 한줄평 10개 중 긍정 비율이 50% 이상인 경우)
(4) 미디어 노출(언론, TV, 라디오 등에서 다룬 경우)
(5) 출판사의 마케팅 지원(광고, 프로모션 등)
(6) 높은 사전 주문량(출간 전에 3000부 이상 주문된 경우)

아래 항목에 대해 해당하면 베스트셀러 가능성 점수에서 1점을 뺍니다.
(1) 부정적인 리뷰(최근 한줄평 10개 중 긍정 비율이 50% 미만인 경우)
(2) 낮은 초기 판매량(출간 후 1주일 이내에 500부 이하 판매된 경우)

[예시1]
아래 내용은 도서 정보입니다. 이 정보를 바탕으로 베스트셀러 점수를 산정하여 베스트셀러 가능성을 예측해주세요.
{
    '번호': 8,
    '제목': '행복한 삶의 비결',
    '이전 베스트셀러 작품 여부': '있음',
    '출간일': '2024-06-01',
    '초기 판매량(부)': 6000,
    '미디어 노출': '있음',
    '마케팅 지원': '있음',
    '사전 주문량(부)': 4000,
    '최근 한줄평': '["정말 유익한 책이에요.", "삶의 질이 높아졌어요.", "다시 읽고 싶은 책입니다.", "친구에게 선물했어요.", "내용이 알차요.", "추천합니다.", "삶의 방향을 잡아줍니다.", "감동적입니다.", "또 읽고 싶어요.", "강추합니다."]'
}

점수 계산 및 분석:
{
  "분석": "이전 베스트셀러 작품이 있으므로 베스트셀러 가능성 점수 1점, 출간 후 1주일 이내에 6000부가 판매되어 베스트셀러 가능성 점수 1점, 최근 한줄평 10개 중 긍정 비율이 50% 이상이므로 베스트셀러 가능성 점수 1점, 미디어 노출이 있어 베스트셀러 가능성 점수 1점, 마케팅 지원이 있어 베스트셀러 가능성 점수 1점, 사전 주문량이 4000부로 베스트셀러 가능성 점수 1점으로 총 6점 베스트셀러 가능성 점수가 있습니다. 이 책은 베스트셀러가 될 가능성이 매우 높습니다.",
  "베스트셀러 점수": 6
}


[예시2]
아래 내용은 도서 정보입니다. 이 정보를 바탕으로 베스트셀러 점수를 산정하여 베스트셀러 가능성을 예측해주세요.
{
    '번호': 23,
    '제목': '어둠의 속삭임',
    '이전 베스트셀러 작품 여부': '없음',
    '출간일': '2024-06-01',
    '초기 판매량(부)': 400,
    '미디어 노출': '없음',
    '마케팅 지원': '없음',
    '사전 주문량(부)': 200,
    '최근 한줄평': '["내용이 지루해요.", "추천하지 않아요.", "전 재밌게 봤어요.", "다시 안 볼 것 같아요.", "별로에요.", "좋지 않아요.", "생각보다 별로였어요.", "기대 이하에요.", "재미없어요.", "괜찮아요."]'
}

점수 계산 및 분석:
{
  "분석": "이전 베스트셀러 작품이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다. 출간 후 1주일 이내에 400부가 판매되어 베스트셀러 가능성 점수에서 1점 감소, 최근 한줄평 10개 중 부정 비율이 높아 베스트셀러 가능성 점수에서 1점 감소, 미디어 노출이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다. 마케팅 지원이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다. 사전 주문량이 200부로 베스트셀러 가능성 점수는 증가하지 않습니다. 총 베스트셀러 가능성 점수는 -2점입니다. 이 책은 베스트셀러가 될 가능성이 낮습니다. 따라서, 판매량 증진을 위한 마케팅 전략이 필요합니다.",
  "베스트셀러 점수": -2
}

[json_request]\n
{\n  \"분석\": \"string\",\n \"베스트셀러 점수\": \"int (-2-6)\",\n}
"""

In [31]:
user_message = f"아래 내용은 도서 정보입니다. 이 정보를 바탕으로 베스트셀러 점수를 산정하여 베스트셀러 가능성을 예측해주세요.\n{book_info}\n\n점수 계산 및 분석:"
user_message

'아래 내용은 도서 정보입니다. 이 정보를 바탕으로 베스트셀러 점수를 산정하여 베스트셀러 가능성을 예측해주세요.\n{\'번호\': 26, \'제목\': \'빛의 자리\', \'이전 베스트셀러 작품 여부\': \'없음\', \'출간일\': \'2024-06-20\', \'초기 판매량(부)\': 350, \'미디어 노출\': \'없음\', \'마케팅 지원\': \'있음\', \'사전 주문량(부)\': 1200, \'최근 한줄평\': \'["감동적입니다.", "추천합니다.", "너무 좋아요.", "다시 읽고 싶어요.", "친구들에게도 추천했어요.", "재밌어요.", "유익해요.", "최고입니다.", "삶의 질이 높아졌어요.", "강추합니다."]\'}\n\n점수 계산 및 분석:'

In [32]:
bestseller_analysis = get_response_gpt(system_message, user_message, model_name='gpt-4o', temperature=temperature, max_tokens=max_tokens, top_p=top_p, use_json_format=True, retry_attempts=3, delay=5)
print(bestseller_analysis)

{
  "분석": "이전 베스트셀러 작품이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다. 출간 후 1주일 이내에 350부가 판매되어 베스트셀러 가능성 점수에서 1점 감소, 최근 한줄평 10개 중 긍정 비율이 50% 이상이므로 베스트셀러 가능성 점수 1점 증가, 미디어 노출이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다. 마케팅 지원이 있어 베스트셀러 가능성 점수 1점 증가, 사전 주문량이 1200부로 베스트셀러 가능성 점수는 증가하지 않습니다. 총 베스트셀러 가능성 점수는 1점입니다. 이 책은 베스트셀러가 될 가능성이 낮지만, 긍정적인 리뷰와 마케팅 지원을 통해 추가적인 판매 촉진이 가능합니다.",
  "베스트셀러 점수": 1
}


## 베스트셀러 가능성이 낮다면(4점 이하) 판매량 증진 전략 제공 

In [40]:
temperature = 0.8
max_tokens = 4096
top_p = 0.75

system_message = "다음 제공되는 도서는 베스트셀러 가능성이 낮아 판매량 증진 전략이 필요합니다. 도서 정보를 확인하여 판매량 증진 전략을 구체적으로 작성해주세요."
user_message = f"[도서 정보]\n{book_info}"
bestseller_analysis = json.loads(bestseller_analysis)

if bestseller_analysis['베스트셀러 점수'] <= 4:
    strategy = get_response_gpt(system_message, user_message, model_name='gpt-4o', temperature=temperature, max_tokens=max_tokens, top_p=top_p, use_json_format=False, retry_attempts=3, delay=5)

print(strategy)

도서 '빛의 자리'의 판매량 증진을 위한 전략을 다음과 같이 제안합니다:

### 1. 타겟 독자층 분석 및 맞춤 마케팅
- **타겟 독자층 설정**: 감동적이고 유익한 책이라는 최근 한줄평을 바탕으로, 감성적이고 자기계발에 관심이 많은 독자층을 주요 타겟으로 설정합니다.
- **맞춤 마케팅 메시지**: "삶의 질을 높이는 감동적인 이야기", "다시 읽고 싶은 책", "친구에게 추천하고 싶은 책" 등의 메시지를 중심으로 마케팅 콘텐츠를 제작합니다.

### 2. 미디어 노출 강화
- **서평 이벤트**: 주요 문학 블로그, 독서 커뮤니티, 서평 전문 사이트와 협력하여 서평 이벤트를 진행합니다. 책을 무료로 제공하고 서평을 작성하게 하여 미디어 노출을 극대화합니다.
- **SNS 마케팅**: 인스타그램, 페이스북, 트위터 등의 SNS 플랫폼에서 감동적인 문구와 책의 핵심 메시지를 담은 이미지 및 영상을 공유합니다. 해시태그 캠페인을 통해 독자들의 자발적인 참여를 유도합니다.

### 3. 저자와의 만남 및 온라인 이벤트
- **저자와의 만남**: 오프라인 서점에서 저자와의 만남, 사인회 등을 개최하여 독자와의 직접적인 소통을 강화합니다.
- **온라인 라이브 이벤트**: 유튜브, 인스타그램 라이브 등을 통해 저자와의 온라인 Q&A 세션을 진행하여 독자들의 관심을 끌어모읍니다.

### 4. 독서 모임 및 북클럽 활용
- **독서 모임 지원**: 지역 독서 모임과 협력하여 '빛의 자리'를 읽고 토론하는 모임을 지원합니다. 책을 무료로 제공하거나 할인 쿠폰을 제공하여 참여를 유도합니다.
- **온라인 북클럽**: 온라인 북클럽을 운영하여 독자들이 책에 대해 토론하고 의견을 나눌 수 있는 플랫폼을 제공합니다.

### 5. 프로모션 및 할인 이벤트
- **초기 구매자 혜택**: 출간 초기 구매자에게 한정판 사은품을 제공하거나 할인 혜택을 제공합니다.
- **교차 프로모션**: 유사한 주제의 다른 베스트셀러와 함께 패키지로 판매하거나, 특정 금액 이상 구매 시 할인 혜

## 최종 출력 

In [41]:
final = f"{book_title}의 베스트셀러 점수는 {bestseller_analysis['베스트셀러 점수']}입니다. {bestseller_analysis['분석']}" + strategy 
print(final)

빛의 자리의 베스트셀러 점수는 1입니다. 이전 베스트셀러 작품이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다. 출간 후 1주일 이내에 350부가 판매되어 베스트셀러 가능성 점수에서 1점 감소, 최근 한줄평 10개 중 긍정 비율이 50% 이상이므로 베스트셀러 가능성 점수 1점 증가, 미디어 노출이 없으므로 베스트셀러 가능성 점수는 증가하지 않습니다. 마케팅 지원이 있어 베스트셀러 가능성 점수 1점 증가, 사전 주문량이 1200부로 베스트셀러 가능성 점수는 증가하지 않습니다. 총 베스트셀러 가능성 점수는 1점입니다. 이 책은 베스트셀러가 될 가능성이 낮지만, 긍정적인 리뷰와 마케팅 지원을 통해 추가적인 판매 촉진이 가능합니다.도서 '빛의 자리'의 판매량 증진을 위한 전략을 다음과 같이 제안합니다:

### 1. 타겟 독자층 분석 및 맞춤 마케팅
- **타겟 독자층 설정**: 감동적이고 유익한 책이라는 최근 한줄평을 바탕으로, 감성적이고 자기계발에 관심이 많은 독자층을 주요 타겟으로 설정합니다.
- **맞춤 마케팅 메시지**: "삶의 질을 높이는 감동적인 이야기", "다시 읽고 싶은 책", "친구에게 추천하고 싶은 책" 등의 메시지를 중심으로 마케팅 콘텐츠를 제작합니다.

### 2. 미디어 노출 강화
- **서평 이벤트**: 주요 문학 블로그, 독서 커뮤니티, 서평 전문 사이트와 협력하여 서평 이벤트를 진행합니다. 책을 무료로 제공하고 서평을 작성하게 하여 미디어 노출을 극대화합니다.
- **SNS 마케팅**: 인스타그램, 페이스북, 트위터 등의 SNS 플랫폼에서 감동적인 문구와 책의 핵심 메시지를 담은 이미지 및 영상을 공유합니다. 해시태그 캠페인을 통해 독자들의 자발적인 참여를 유도합니다.

### 3. 저자와의 만남 및 온라인 이벤트
- **저자와의 만남**: 오프라인 서점에서 저자와의 만남, 사인회 등을 개최하여 독자와의 직접적인 소통을 강화합니다.
- **온라인 라이브 이벤트**: 유튜브, 인스타그램 라이브 등을 통해 저자와의 온라인 Q&A 세션을